In [1]:
import requests
import pandas as pd
from datetime import datetime

# make_params()함수에 사용하는 내부 함수. 날짜를 변경하는 함수
def date_change(date):
    nowdate = datetime.strptime(date, "%Y-%m-%d")
    ldate = (nowdate.replace(month=12).replace(day=31)).strftime("%Y%m%d")
    nowdate_str = nowdate.strftime("%Y%m%d")
    return nowdate_str, ldate

# 변경된 날짜로 파라미터를 생성하는 함수
def make_params(date):
    startDt, endDt = date_change(date)
    result = {
    'serviceKey' : 'fF/Ekuf8awXAsiiMYAivKxzIu1Hyl76PXGGqz88Dvs7sMLoKbSaWzAxF2//kihhIurY9PYr5kxC6lEvw8mDC4Q==',
    'pageNo' : '1',
    'numOfRows' : '365',
    'dataType' : 'XML',
    'dataCd' : 'ASOS',
    'dateCd' : 'DAY',
    'startDt' : startDt,
    'endDt' : endDt,
    'stnIds' : '108'
    }
    return result

# api로 데이터 가져와서 적당히 정제하는 코드
def get_api(url, params):
    response = requests.get(url, params=params)
    contents = response.content
    decode_xml= contents.decode('UTF-8')
    return decode_xml.replace('/','').replace('>','').split('<')

# data_filter() 함수에 사용하는 내부 함수. 빈 문자열 제거하는 함수
def get_data(list,char):
    result = []
    for i in list:
        char_len = len(char)
        if char in i:
            result.append(i[char_len:])
    return [item for item in result if item and item.strip()]

# 원하는 데이터(딕셔너리에 정의된 키:값 사용)만 추출하는 함수
def data_filter(data,dict):
    result = {}
    for a,b in dict.items():
        result = {**result,**{a:get_data(data,b)}}
    return result


url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
dict = {
    '지점번호' : "stnId",
    '지점명' : 'stnNm',
    '시간' : 'tm',
    '평균 기온' : 'avgTa',
    '평균 지면온도' : 'avgTs'
}
date_list = ["2014-01-01","2015-01-01","2016-01-01","2017-01-01","2018-01-01","2019-01-01","2020-01-01","2021-01-01","2022-01-01","2023-01-01"]


for date in date_list:
    params = make_params(date)
    data = get_api(url, params)
    filt_Data = data_filter(data,dict)
    df = pd.DataFrame(filt_Data)
    print(df)

    지점번호 지점명          시간 평균 기온 평균 지면온도
0    108  서울  2014-01-01   4.0     0.8
1    108  서울  2014-01-02   1.5     0.0
2    108  서울  2014-01-03   2.3     0.1
3    108  서울  2014-01-04  -0.1    -0.2
4    108  서울  2014-01-05  -0.8    -0.7
..   ...  ..         ...   ...     ...
360  108  서울  2014-12-27  -1.8    -1.8
361  108  서울  2014-12-28  -0.8    -1.4
362  108  서울  2014-12-29   2.9     0.3
363  108  서울  2014-12-30   1.9     0.7
364  108  서울  2014-12-31  -1.9     0.2

[365 rows x 5 columns]
    지점번호 지점명          시간 평균 기온 평균 지면온도
0    108  서울  2015-01-01  -7.7    -4.1
1    108  서울  2015-01-02  -6.0    -3.6
2    108  서울  2015-01-03  -2.7    -2.6
3    108  서울  2015-01-04   2.5     1.1
4    108  서울  2015-01-05   3.7     1.1
..   ...  ..         ...   ...     ...
360  108  서울  2015-12-27  -5.7    -0.6
361  108  서울  2015-12-28  -5.7    -2.1
362  108  서울  2015-12-29  -3.2    -1.8
363  108  서울  2015-12-30   0.3    -0.3
364  108  서울  2015-12-31   0.7     0.6

[365 rows x 5 columns]
    지점번호 지점명    